# **NN**


Bu not defteri, elektrikli araç enerji tüketimi için geliştirilen NN tabanlı tahmin modelinin veri hazırlama, eğitim ve değerlendirme adımlarını ayrıntılı olarak sunar.



## **1. Kütüphaneleri Yükleme**

In [65]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import joblib
import random

In [120]:
# Seed ayarı (tekrar üretilebilirlik için)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## **2. Veri Yükleme ve Ön İnceleme**

In [121]:
uploaded = files.upload()
df = pd.read_csv("ev_dataset_multi_target.csv")
DATA_PATH = "ev_dataset_multi_target.csv"
df.head()

Saving ev_dataset_multi_target.csv to ev_dataset_multi_target (3).csv


,CELL_V_AVG,Torque_Measured,MotorTemperature,DC_Link_Voltage,DC_Link_Current,Experiment_ID,Average_Velocity,Load,Direction,Season,...,speed_ms,acc_ms2,power_W,cell_voltage_diff,SoC,soc_net_per_s,soc_net_per_s_smooth5,soc_net_per_s_smooth10,soc_net_per_s_30s,soc_net_per_s_60s
0,3316.50,12.56,84.68,49.687500,14.17,Experiment_1,15,Loaded,CISAR,Summer,...,0.044935,0.000000,704.071875,7.54,60.030,0.000,0.0028,0.0048,0.0,0.0
1,3313.15,24.26,84.52,49.593125,30.35,Experiment_1,15,Loaded,CISAR,Summer,...,0.488215,0.443281,1505.151344,7.47,60.022,0.008,0.0028,0.0048,0.0,0.0
2,3311.55,22.00,84.47,49.530625,34.48,Experiment_1,15,Loaded,CISAR,Summer,...,0.968013,0.479798,1707.815950,7.83,60.020,0.002,0.0028,0.0048,0.0,0.0
3,3309.89,22.52,84.65,49.482500,41.73,Experiment_1,15,Loaded,CISAR,Summer,...,1.348039,0.380026,2064.904725,7.84,60.020,-0.000,0.0040,0.0048,0.0,0.0
4,3308.00,23.06,84.63,49.441875,47.54,Experiment_1,15,Loaded,CISAR,Summer,...,1.649832,0.301792,2350.466737,7.53,60.016,0.004,0.0032,0.0048,0.0,0.0


In [122]:
print("Eksik veri durumu:")
print(df.isnull().sum())

Eksik veri durumu:
CELL_V_AVG                0
Torque_Measured           0
MotorTemperature          0
DC_Link_Voltage           0
DC_Link_Current           0
Experiment_ID             0
Average_Velocity          0
Load                      0
Direction                 0
Season                    0
slope                     0
speed_ms                  0
acc_ms2                   0
power_W                   0
cell_voltage_diff         0
SoC                       0
soc_net_per_s             0
soc_net_per_s_smooth5     0
soc_net_per_s_smooth10    0
soc_net_per_s_30s         0
soc_net_per_s_60s         0
dtype: int64


## **3. Özellik Mühendisliği (Feature Engineering)**

In [124]:

# Dataseti oku
df = pd.read_csv("ev_dataset_multi_target.csv")

to_encode = ["Load", "Direction", "Season"]

# One-hot encoding -> object sütunları tamamen sayısala çevir
df_encoded = pd.get_dummies(df, columns=to_encode, drop_first=True, dtype=int)

print(df_encoded.dtypes)


CELL_V_AVG                float64
Torque_Measured           float64
MotorTemperature          float64
DC_Link_Voltage           float64
DC_Link_Current           float64
Experiment_ID              object
Average_Velocity            int64
slope                     float64
speed_ms                  float64
acc_ms2                   float64
power_W                   float64
cell_voltage_diff         float64
SoC                       float64
soc_net_per_s             float64
soc_net_per_s_smooth5     float64
soc_net_per_s_smooth10    float64
soc_net_per_s_30s         float64
soc_net_per_s_60s         float64
Load_Unloaded               int64
Direction_MMF               int64
Season_Winter               int64
dtype: object


## **4. Gereksiz Sütunları Kaldırma**

In [125]:
drop_targets = {
    "soc_net_per_s",
    "soc_net_per_s_smooth10",
    "soc_net_per_s_30s",
    "soc_net_per_s_60s"
}
df_encoded.drop(columns=drop_targets ,inplace=True )
df_encoded.head()

,CELL_V_AVG,Torque_Measured,MotorTemperature,DC_Link_Voltage,DC_Link_Current,Experiment_ID,Average_Velocity,slope,speed_ms,acc_ms2,power_W,cell_voltage_diff,SoC,soc_net_per_s_smooth5,Load_Unloaded,Direction_MMF,Season_Winter
0,3316.50,12.56,84.68,49.687500,14.17,Experiment_1,15,0.0,0.044935,0.000000,704.071875,7.54,60.030,0.0028,0,0,0
1,3313.15,24.26,84.52,49.593125,30.35,Experiment_1,15,0.0,0.488215,0.443281,1505.151344,7.47,60.022,0.0028,0,0,0
2,3311.55,22.00,84.47,49.530625,34.48,Experiment_1,15,0.0,0.968013,0.479798,1707.815950,7.83,60.020,0.0028,0,0,0
3,3309.89,22.52,84.65,49.482500,41.73,Experiment_1,15,0.0,1.348039,0.380026,2064.904725,7.84,60.020,0.0040,0,0,0
4,3308.00,23.06,84.63,49.441875,47.54,Experiment_1,15,0.0,1.649832,0.301792,2350.466737,7.53,60.016,0.0032,0,0,0


## **5. Veriyi Eğitim, Doğrulama ve Test Setlerine Ayırma**

Veri sızıntısını önlemek amacıyla, aynı rotaya ait örneklerin setler arası karışmasını engelleyen GroupShuffleSplit yöntemiyle veri, eğitim–doğrulama–test kümelerine ayrıştırılmıştır.

In [126]:
groups = df_encoded["Experiment_ID"]

X = df_encoded.drop(columns=[ "Experiment_ID"])
y = df_encoded["soc_net_per_s_smooth5"]


def group_split(X, y, groups, test_size=0.15, val_size=0.15, rs=42):
    """
    X, y ve groups'u alır; önce TEST setini, sonra kalan içinden VALIDATION setini ayırır.
    NOT: Aynı 'vehicle_id'ye ait TÜM satırlar ya train/val tarafında kalır ya da testte olur;
         asla bir araca ait satırlar birden fazla parçaya bölünmez.
    """
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=rs)
    i_tr, i_te = next(gss.split(X, y, groups))
    gss2 = GroupShuffleSplit(n_splits=1, test_size=val_size/(1-test_size), random_state=rs)
    j_tr, j_val = next(gss2.split(X.iloc[i_tr], y.iloc[i_tr], groups.iloc[i_tr]))
    tr_idx = X.iloc[i_tr].index[j_tr]; val_idx = X.iloc[i_tr].index[j_val]; te_idx = X.index[i_te]
    return tr_idx, val_idx, te_idx

train_idx, val_idx, test_idx = group_split(X, y, groups)
X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_val,   y_val   = X.loc[val_idx],   y.loc[val_idx]
X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
print(f"Split -> train:{len(train_idx)}, val:{len(val_idx)}, test:{len(test_idx)}")
print("\nEğitim için kullanılacak özelliklerin bilgi özeti:")
X_train.info()

Split -> train:10224, val:2274, test:2159

Eğitim için kullanılacak özelliklerin bilgi özeti:
<class 'pandas.core.frame.DataFrame'>
Index: 10224 entries, 0 to 14656
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CELL_V_AVG             10224 non-null  float64
 1   Torque_Measured        10224 non-null  float64
 2   MotorTemperature       10224 non-null  float64
 3   DC_Link_Voltage        10224 non-null  float64
 4   DC_Link_Current        10224 non-null  float64
 5   Average_Velocity       10224 non-null  int64  
 6   slope                  10224 non-null  float64
 7   speed_ms               10224 non-null  float64
 8   acc_ms2                10224 non-null  float64
 9   power_W                10224 non-null  float64
 10  cell_voltage_diff      10224 non-null  float64
 11  SoC                    10224 non-null  float64
 12  soc_net_per_s_smooth5  10224 non-null  float64
 13  Load_Unloaded    

StandardScaler, her sütunu ortalaması 0 ve standart sapması 1 olacak şekilde ölçekleyen bir normalizasyon aracıdır. Böylece tüm özellikler benzer ölçekte olur ve modelin farklı feature’lara dengeli yaklaşması sağlanır. Veri sızıntısını (data leakage) önlemek için fit işlemi yalnızca eğitim setinde yapılır; doğrulama ve test setlerine sadece transform uygulanır. Aksi halde, bu setlerin istatistiklerini öğrenmiş oluruz ve model eğitimde görmemesi gereken bilgiyi kullanır.

In [127]:
#Normalizasyon
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [128]:
#Dataset Sınıfı
class EnergyDataset(Dataset):
    # Veriyi tensör haline getirip DataLoader ile batch’ler halinde beslememize olanak sağlar.
    def __init__(self, X, y):
        self.X = torch.tensor(X.values if hasattr(X, 'values') else X, dtype=torch.float32)
        self.y = torch.tensor(y.values if hasattr(y, 'values') else y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [129]:
train_loader = DataLoader(EnergyDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(EnergyDataset(X_val, y_val), batch_size=32, shuffle=False)
test_loader = DataLoader(EnergyDataset(X_test, y_test), batch_size=32, shuffle=False)

* **Mimari**: Giriş → 256 → 128 → 64  (daralan MLP).

* **Aktivasyon (ReLU)**: Negatif girişleri sıfırlar, pozitif bölgede sabit gradyan ile vanishing gradient riskini azaltır; normalleştirilmiş verilerle hızlı yakınsama sağlar.

* **Batch Normalization:** Katman içi dağılımı düzenler; daha hızlı/istikrarlı eğitim ve daha iyi genelleme.

* **Dropout:** Rastgele bağlantı kapatma ile co-adaptation’ı azaltır; overfitting riskini düşürür.

* **Birlikte etkisi:** ReLU + BatchNorm + Dropout kombinasyonu, enerji tüketimi tahmininde düşük hata ve yüksek R² için dengeli bir öğrenme sunar.

In [130]:
#MLP modeli
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.15),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

In [131]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(X_train.shape[1]).to(device)

**MSELoss (Mean Squared Error):** Regresyonda yaygın kayıp fonksiyonudur. Tahmin–gerçek farkının karesi alınır ve ortalaması hesaplanır. Büyük hataları daha fazla cezalandırır.

**Adam optimizer:** Momentum ve adaptif öğrenme oranı fikirlerini birleştirir; genelde hızlı ve kararlı yakınsama sağlar.

**ReduceLROnPlateau:** İzlenen doğrulama metriği (ör. val loss) patience süresi boyunca iyileşmezse öğrenme oranını factor katsayısı kadar düşürür; böylece daha iyi minimumlara inmek kolaylaşır.

In [132]:
#Loss, optimizer, schedular
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 mode='min',  # validation kaybı minimuma inmeye çalışılır.
                                                 factor=0.1,  # learning rate, iyileşme olmazsa mevcut değerinin %10’una düşürülür.
                                                 patience=5,  # 5 epoch boyunca iyileşme olmazsa düşürme işlemi yapılır.
                                                 )

In [135]:

torch.set_grad_enabled(True)

model.train()
xb_chk, yb_chk = next(iter(train_loader))
xb_chk, yb_chk = xb_chk.to(device).float(), yb_chk.to(device).float()
out_chk = model(xb_chk)
print("Sanity | out.requires_grad:", out_chk.requires_grad)
loss_chk = criterion(out_chk, yb_chk)
print("Sanity | loss.requires_grad:", loss_chk.requires_grad)
assert any(p.requires_grad for p in model.parameters()), "Model parametreleri donmuş (requires_grad=False)!"

# 1) Eğitim döngüsü
best_val = float('inf')
patience, wait = 10, 0

for epoch in range(25):
    # ---- TRAIN ----
    model.train()
    torch.set_grad_enabled(True)

    for xb, yb in train_loader:
        xb = xb.to(device).float()
        yb = yb.to(device).float()
        optimizer.zero_grad(set_to_none=True)
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    # ---- VALIDATION ----
    model.eval()
    torch.set_grad_enabled(False)
    y_true, y_pred = [], []

    for xb, yb in val_loader:
        xb = xb.to(device).float()
        yb = yb.to(device).float()
        preds = model(xb)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

    val_mae = mean_absolute_error(y_true, y_pred)
    val_rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    val_r2 = r2_score(y_true, y_pred)
    scheduler.step(val_mae)

    print(f"Epoch {epoch+1} | Val MAE: {val_mae:.4f} | RMSE: {val_rmse:.4f} | R²: {val_r2:.4f}")

    if val_mae < best_val:
        best_val = val_mae
        torch.save(model.state_dict(), "best_mlp.pt")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping.")
            break
torch.set_grad_enabled(True)

Sanity | out.requires_grad: True
Sanity | loss.requires_grad: True
Epoch 1 | Val MAE: 0.0006 | RMSE: 0.0008 | R²: 0.9736
Epoch 2 | Val MAE: 0.0008 | RMSE: 0.0011 | R²: 0.9503
Epoch 3 | Val MAE: 0.0008 | RMSE: 0.0009 | R²: 0.9642
Epoch 4 | Val MAE: 0.0010 | RMSE: 0.0013 | R²: 0.9339
Epoch 5 | Val MAE: 0.0004 | RMSE: 0.0006 | R²: 0.9869
Epoch 6 | Val MAE: 0.0003 | RMSE: 0.0006 | R²: 0.9870
Epoch 7 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9938
Epoch 8 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9921
Epoch 9 | Val MAE: 0.0006 | RMSE: 0.0008 | R²: 0.9741
Epoch 10 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9946
Epoch 11 | Val MAE: 0.0004 | RMSE: 0.0005 | R²: 0.9883
Epoch 12 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9936
Epoch 13 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9923
Epoch 14 | Val MAE: 0.0006 | RMSE: 0.0006 | R²: 0.9836
Epoch 15 | Val MAE: 0.0003 | RMSE: 0.0005 | R²: 0.9893
Epoch 16 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9933
Epoch 17 | Val MAE: 0.0003 | RMSE: 0.0004 | R²: 0.9921
Epoch 1

torch.autograd.grad_mode.set_grad_enabled(mode=True)

## **6. Model Değerlendirme**


Modelin performansı, doğrulama ve test kümeleri üzerinde üretilen tahminler aracılığıyla değerlendirilmiş; başarı, Ortalama Kare Hata (MSE), Kök Ortalama Kare Hata (RMSE) ve Belirlilik Katsayısı (R²) metrikleriyle nicel olarak raporlanmıştır.

In [136]:
# ===== 9. Test değerlendirme =====
model.load_state_dict(torch.load("best_mlp.pt"))
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb.to(device))
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

test_mae = mean_absolute_error(y_true, y_pred)
test_rmse = math.sqrt(mean_squared_error(y_true, y_pred))
test_r2 = r2_score(y_true, y_pred)
print(f"Epoch {epoch+1} | TEST MAE: {test_mae:.4f} | RMSE: {test_rmse:.4f} | R²: {test_r2:.4f}")

Epoch 25 | TEST MAE: 0.0002 | RMSE: 0.0003 | R²: 0.9972
